# Advanced - Custom features

This notebook requires Python coding knowledge. It is also recommended to first read the notebooks [Pycellin data structure](./Pycellin%20data%20structure.ipynb) (stub notebook for now) and [Managing features](./Managing%20features.ipynb) (mostly done).

In this notebook, we will cover how to augment a model with a user-defined feature.

- describe the new feature  
- implement the way to compute it (i.e. its calculator)
- add the new feature and its associated calculator to the model  
- update the model so the new feature values are computed and stored properly for all relevant elements of the lineages

In [1]:
import pycellin as pc

In [2]:
# Path to the TrackMate XML file.
xml_path = "../sample_data/Ecoli_growth_on_agar_pad.xml"

# Parse the XML file and create a Pycellin Model object
# that contains all the data from the XML file.
model = pc.load_TrackMate_XML(xml_path)

# We can display basic information about this model.
print(model)
print(f"This model contains {model.data.number_of_lineages()} lineages:")
for lin_ID, lineage in model.data.cell_data.items():
    print(f"- ID {lin_ID}: {lineage}")

Model with 3 lineages.
This model contains 3 lineages:
- ID 0: CellLineage named 'Track_0' with 152 nodes and 151 edges
- ID 1: CellLineage named 'Track_1' with 189 nodes and 188 edges
- ID 2: CellLineage named 'Track_2' with 185 nodes and 184 edges


## Definition of the new feature 

A new feature is defined by creating a new instance of the `Feature` class, with 6 fields that describe the feature:

**name**  
The name/identifier of the feature. This will be the name of the associated Python variable and must follow variable naming rules (no punctuations except underscore, no whitespaces, no numbers).

**description**  
A concise description of the feature.

**lineage_type**    
Either `CellLineage` or `CycleLineage` depending on which type of lineage your feature is related to. See [Pycellin data structure](./Pycellin%20data%20structure.ipynb) if in doubt.

**provenance**  
Where does the feature come from? For imported features, Pycellin uses the name of the tool the data was imported from. For custom features, you can use `custom`, your initials or whatever works for you. This field is useful for traceability (e.g. reopening a model after a long time, sharing a model with other people...).

**data_type**   
Python type of the feature values, e.g. `int`, `bool`...

**unit**    
Unit of the feature values, e.g. `µm`, `min`, `cell`...


All these fields are strings. They are mandatory except `unit` which will be set to `None` if not provided.

Here is an example on a basic feature, the age of the cells:

In [3]:
age_feat = pc.Feature(
    "age",  # name
    "Age of the cell since the beginning of the lineage",  # description
    "CellLineage",  # lineage_type
    "custom",  # provenance
    "int",  # data_type
    "frame",  # unit
)

You can then access the different fields of the feature:

In [4]:
print(f"{age_feat.name}: {age_feat.description} in {age_feat.unit}(s)")

age: Age of the cell since the beginning of the lineage in frame(s)


## Calculators

In Pycellin, a calculator is a class that defines and structures how to compute a specific feature. A feature cannot be associated to more than one calculator. As we will see, calculators can be very simple with just a few lines of code, or really complex. 

Currently, imported features do not have calculators. This means that imported features cannot be recomputed unless you define a calculator yourself and associate it with the imported feature.  
Pycellin features come with their own calculators and are stored within the `pycellin.graph.features` subpackage. This is a useful place to look for real examples of calculators.

When you want to compute something that is not provided by Pycellin nor imported from a tracking tool, you need to define your own calculator. Here is the stripped-down generic structure of a calculator:

```python
class MyCalculator(CalculatorToInheritFrom):

    def compute(self, *args):
        # Code to compute the value of the feature for a single element, 
        # either a node, an edge or a lineage graph.        
        return feature_value
```

The calculator class must ALWAYS define at least a `compute()` method. In more complex calculators, you can also have an `__init__()`method (see the [In case of additional arguments](#in-case-of-additional-arguments) section).

With our `age` feature previously defined, the calculator could be:

In [5]:
class AgeCalculator(pc.NodeGlobalFeatureCalculator):

    def compute(self, data: pc.Data, lineage: pc.CellLineage, noi: int) -> int:
        root = lineage.get_root()
        return lineage.nodes[noi]["frame"] - lineage.nodes[root]["frame"]

### How to choose the calculator to inherit from?

In the example above, we have built our `AgeCalculator` by inheriting from `NodeGlobalFeatureCalculator`. But there are several `FeatureCalculator` base classes you can inherit from, represented in orange in the  calculators tree below. Arrows represent inheritance. 

![Calculators tree scheme](./imgs/Pycellin_calculators_scheme.png)

To know which calculator to use, you need to answer 2 questions:
1. Where do you want to store the feature values? On nodes, edges, or lineage graphs?
2. What information do you need to compute your new feature?

Question 1 is already answered in the previous part (not currently, but will be in next update since the feature type (node, edge, lineage) is going to be one of the mandatory field of a Feature). 

Question 2 requires to think about **the reach of your feature**: local vs global.  
A local feature is a feature that only needs data from the element it will be stored in. For example, cell area would be a local feature since it only needs information about the shape of the cell itself to be computed. But some features require data from several elements or maybe from the whole lineage. In that case they are global. A simple example is our `AgeCalculator`. To compute the age of a cell you need to know when the first cell of the lineage appeared. So you need to access the data stored into 2 different cells: the cell of interest and the root of the lineage.  

These 2 examples, area and age, are examples of node features, so their calculators would have to inherit from `NodeLocalFeatureCalculator` and `NodeGlobalFeatureCalculator` respectively.  
Computing the speed of a cell between 2 frames would need an `EdgeLocalFeatureCalculator` since only the position of the 2 cells forming the edge is needed.  
For lineage, mean cell speed is an example of a feature based on a `LineageLocalFeatureCalculator`. It requires information from all the edges of the current lineage, but nothing from other lineages.

## Adding the new feature and its calculator to the model

Once the feature and its calculator are defined, they need to be linked together and added to the model. This is done with the `add_custom_feature()` method:

```python
model.add_custom_feature(
    my_feature,  # instance of Feature defined above
    MyCalculator,  # Calculator class defined above (not instanciated!)
)
```

For our `age` example:

In [6]:
model.add_custom_feature(age_feat, AgeCalculator)

## Updating the model

When a new feature is added with `add_custom_feature()`, a full update of the model is automatically planned.  

To actually run the update that will compute and add the feature values for the newly added custom feature, you simply need to call the `update()` method on the model:

In [7]:
model.update()

During the update, all the features associated with a calculator will be computed or recomputed. See [Managing features](./Managing%20features.ipynb) for more information.

We can check that our custom `age` feature has been added to all nodes by plotting a lineage and hovering over its cells:

In [8]:
lin0 = model.data.cell_data[0]
lin0.plot(node_hover_features=["cell_ID", "age"])

## Removing a custom feature

A custom feature can be removed from a model like any other features:

In [9]:
model.remove_feature("age", "node")

See [Managing features](./Managing%20features.ipynb) for more information.

## Examples

### Importance of `compute()` signature

The signature of the `compute` method must be respected, even if one of the argument is not needed to compute the feature.  
Here is a very simple example with the `node_ID_parity` feature:

In [10]:
feat_incorrect = pc.Feature(
    name="node_ID_parity_incorrect",
    description="Parity of the node ID",
    lineage_type="CellLineage",
    provenance="Pycellin",
    data_type="int",
)


class ParityCalculator_incorrect(pc.NodeLocalFeatureCalculator):

    def compute(self, noi):
        if noi % 2 == 0:
            return True
        else:
            return False

In [11]:
model.add_custom_feature(feat_incorrect, ParityCalculator_incorrect)
model.update()

TypeError: ParityCalculator_incorrect.compute() takes 2 positional arguments but 3 were given

The update cannot be done because the custom calculator does not abide by `compute()` signature: it is missing the `lineage` argument. The signature is enforced to provide a relatively simple but consistent API. The arguments expected by `compute()` depend on the inherited calculator, i.e. both on the feature type (node, edge or lineage) and the feature reach (local or global). 

Here is the correct way to define the `node_ID_parity` feature:

In [12]:
feat_correct = pc.Feature(
    name="node_ID_parity_correct",
    description="Parity of the node ID",
    lineage_type="CellLineage",
    provenance="Pycellin",
    data_type="int",
)


class ParityCalculator_correct(pc.NodeLocalFeatureCalculator):

    def compute(self, lineage, noi):
        # `lineage` is unused here, but it is still a required argument.
        if noi % 2 == 0:
            return True
        else:
            return False

In [13]:
model.add_custom_feature(feat_correct, ParityCalculator_correct)

# We clean the model by removing the improper feature,
# otherwise we won't be able to update the model.
model.remove_feature("node_ID_parity_incorrect", "node")
model.update()

# Checking that the feature was correctly added.
print(lin0.nodes[8993]["node_ID_parity_correct"])
print(lin0.nodes[9014]["node_ID_parity_correct"])

False
True


### In case of additional arguments

Sometimes, your custom feature needs additional arguments to be computed, for example specific information from the model, like pixel size or the time elapsed between frames.  
In that case, you need to overwrite the `__init__` method of the `FeatureCalculator` parent class. Here is a snippet of this class:

```python
# Code from pycellin.classes.feature_calculator.py

from abc import ABC

class FeatureCalculator(ABC):
    """
    Abstract class to compute feature values.
    """

    def __init__(self, feature: pc.Feature):
        self.feature = feature
```

So the `__init__` of your calculator must:
1. take your new `Feature` as argument;
2. call the parent calculator own `__init__` on the `Feature` with `super().__init__(feature)`.

Below, a concrete example on the relative age of a cell, given in time units:

In [14]:
age_feat2 = pc.Feature(
    "relative_age",
    "Age of the cell since the beginning of the current cell cycle",
    "CellLineage",
    "custom",
    "float",
    model.get_time_step(),
)


class RelativeAgeCalculator(pc.NodeGlobalFeatureCalculator):

    def __init__(self, feature: pc.Feature, time_step: float):
        # Call the parent __init__ method.
        super().__init__(feature)
        # Do whatever you want with the additional argument(s).
        # Usually you store them in the object to be able to use them
        # in the `compute` method.
        self.time_step = time_step

    def compute(self, data: pc.Data, lineage: pc.CellLineage, noi: int):
        # As said before, the signature of the `compute` method must be respected,
        # even if we don't use the `data` argument here.
        first_cell = lineage.get_cell_cycle(noi)[0]
        first_cell_frame = lineage.nodes[first_cell]["frame"]
        current_cell_frame = lineage.nodes[noi]["frame"]
        age_in_frame = current_cell_frame - first_cell_frame
        # Here we use the additional time step argument.
        age_in_time = age_in_frame * self.time_step
        return age_in_time

After that, the feature can be added to the model as usual.

**QUESTION FOR MARIE: which syntax do you prefer?**

Syntax A (current one):

In [15]:
model.add_custom_feature(
    age_feat2, RelativeAgeCalculator, model.get_time_step())

Syntax B (leaves the instanciation to the user):

In [ ]:
calc = RelativeAgeCalculator(age_feat2, model.get_time_step())
model.add_custom_feature(calc)

In [17]:
model.update()
lin0.plot(node_hover_features=["cell_ID", "relative_age"])

### Growth rate: a cycle lineage feature

In [19]:
gr_feat = pc.Feature(
    "growth_rate",
    (
        "Growth rate of the bacterium during a cell cycle, "
        "i.e. (area_at_div_time - area_at_birth_time) / division_time"
    ),
    "CycleLineage",
    "custom",
    "float",
    f"{model.get_space_unit()}2/{model.get_time_unit()}",
)


class GrowthRateCalculator(pc.NodeGlobalFeatureCalculator):

    def compute(self, data, lineage, noi):
        cell_lin = data.cell_data[lineage.graph["cycle_lineage_ID"]]
        birth_cell = lineage.nodes[noi]["cells"][0]
        area_birth = cell_lin.nodes[birth_cell]["area"]
        div_cell = lineage.nodes[noi]["cells"][-1]
        area_div = cell_lin.nodes[div_cell]["area"]
        div_time = lineage.nodes[noi]["division_time"]
        return (area_div - area_birth) / div_time

In [20]:
model.add_cycle_data()
# Division time is a Pycellin feature, and it is needed to compute the growth rate.
model.add_division_time(in_time_unit=True)
model.add_custom_feature(gr_feat, GrowthRateCalculator)
model.update()
model.data.cycle_data[0].plot(node_hover_features=["cycle_ID", "growth_rate"])